In [114]:
import json
import os
import django
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link,PermanentSubject,SubjectChoices
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model
import datetime
import pandas as pd
import string

In [37]:
file_path='/home/manav1011/Documents/SMARTROLL_SSIP_2023/SMARTROLL/notebooks/sheets/electives.xlsx'

In [174]:
sheet_names = pd.ExcelFile(file_path).sheet_names
ML = pd.read_excel(file_path, sheet_name='ML')
BDA = pd.read_excel(file_path, sheet_name='BDA')
IS = pd.read_excel(file_path, sheet_name='IS')
MCWC = pd.read_excel(file_path, sheet_name='MCWC')
CC = pd.read_excel(file_path, sheet_name='CC')
MAD = pd.read_excel(file_path, sheet_name='MAD')
DF = pd.read_excel(file_path, sheet_name='DF')

In [175]:
def update_student_subjects(df, subject_name,student_subjects):
    for _, row in df.iterrows():
        student_key = (row['enrollment'], row['name'])  # Create a tuple of (enrollment, name)
        if student_key in student_subjects:
            student_subjects[student_key].append(subject_name)
        else:
            student_subjects[student_key] = [subject_name]


In [176]:
student_subjects = {}
update_student_subjects(ML, 'ML',student_subjects)
update_student_subjects(BDA, 'BDA',student_subjects)
update_student_subjects(IS, 'IS',student_subjects)
update_student_subjects(MCWC, 'MCWC',student_subjects)
update_student_subjects(CC, 'CC',student_subjects)
update_student_subjects(MAD, 'MAD',student_subjects)
update_student_subjects(DF, 'DF',student_subjects)

In [201]:
min_div_size = 60
max_div_size = 90
def calculate_divisions(student_count):    
    min_divisions = (student_count + max_div_size - 1) // max_div_size
    max_divisions = student_count // min_div_size        
    for divisions in range(min_divisions, max_divisions + 1):
        div_size = student_count // divisions
        if min_div_size <= div_size <= max_div_size:
            return divisions        
    return min_divisions

In [202]:
total_student_count = len(student_subjects.keys())
approx_division_count = calculate_divisions(total_student_count)

In [203]:
subject_groups = {}
for student, subject_group in student_subjects.items():
    # Convert subject_group to a tuple if it isn't one already
    subject_group_tuple = tuple(subject_group)
    if subject_group_tuple not in subject_groups:
        subject_groups[subject_group_tuple] = [student]
    else:
        subject_groups[subject_group_tuple].append(student)

In [204]:
divisions = {}
for i in range(approx_division_count):
    divisions[string.ascii_uppercase[i]] = []

In [205]:
remaining_students_count = total_student_count
for division in divisions:    
    # check the count of students in current division
    students_count_in_div = len(divisions[division])
    capacity_remained_in_div = max_div_size-students_count_in_div
    if capacity_remained_in_div >= remaining_students_count:
        # insert all the remaining students in current div
        for subject_group,students in subject_groups.items():
            divisions[division] += students
    else:
        # iterate through the subject groups
        # check if any available subject group has the student count greater than the max_div_count
        subject_group_gt_max_div_count= list(filter(lambda subject_group: len(subject_groups[subject_group]) > max_div_size,subject_groups))
        if subject_group_gt_max_div_count:
            # move the first max_div_count students to this division
            divisions[division] = subject_groups[subject_group_gt_max_div_count[0]][:max_div_size]
            subject_groups[subject_group_gt_max_div_count[0]] = subject_groups[subject_group_gt_max_div_count[0]][max_div_size:]
            # also reduce the remaining student count            
            remaining_students_count -= max_div_size
            # now this div's full so move let's move to next div            
            continue    

In [206]:
# Now to sort the students according to their enrollment in each divisions
for division in divisions:
    divisions[division]=sorted(divisions[division], key=lambda student: student[0])
divisions

{'A': [(200280107145, 'Prajapati Jay Kumar Dashrathbhai '),
  (210280107001, 'Ratadiya Mayur Devsibhai '),
  (210280107003, 'Taral Bambhania'),
  (210280107004, 'Darshil Nareshbhai Patel'),
  (210280107005, 'Akshar Chauhan'),
  (210280107006, 'Patel Nisarg Surendrakumar'),
  (210280107007, 'DOBARIYA HAPPY RAMNIKBHAI '),
  (210280107008, 'Meet Soni '),
  (210280107009, 'Nirali Darji'),
  (210280107010, 'Devmurari Maulik Vijaybhai'),
  (210280107011, 'Justin George'),
  (210280107012, 'Patel Dhruvit Vasantbhai '),
  (210280107013, 'Halpati Divya DipakKumar '),
  (210280107014, 'Jiyani Hasti Mukeshbhai'),
  (210280107015, 'Astha Hareshbhai Viradiya '),
  (210280107016, 'Kaman Devanshi KaushikKumar '),
  (210280107017, 'Naik Chinmay Dharmesh'),
  (210280107018, 'Patel Chirag Sunilbhai'),
  (210280107019, 'PRAJAPATI HARSH DHIRENDRAKUMAR '),
  (210280107020, 'KERALIYA KARN NAROTTAMBHAI'),
  (210280107022, 'Henish Kantibhai Naliyadhara '),
  (210280107024, 'Jha Kunjbihari Vinodanand '),
  (21